In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


messages = [line.rstrip() for line in open('spam_ham_dataset.csv')]
print(len(messages))

# # making dataframe
# df = pd.read_csv("spam_ham_dataset.csv")

# # output the dataframe
# print(df.head())

messages = pd.read_csv('spam_ham_dataset.csv', sep=',', names=['Ignore', 'Label', 'message', 'label_num'])
messages.head()

101594


,Ignore,Label,message,label_num
0,NaN,label,text,label_num
1,605.0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
2,2349.0,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
3,3624.0,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
4,4685.0,spam,"Subject: photoshop , windows , office . cheap ...",1


In [2]:
messages['label_num'].head()

0    label_num
1            0
2            0
3            0
4            1
Name: label_num, dtype: object

**Bag of words approch filtering out stop words and TF/IDF Approch**

In [3]:
!pip install nltk

In [4]:
import string

import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
def text_process(mess):
    #remove punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)

    #Now remove any stop words
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Vectorization

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

print(len(bow_transformer.vocabulary_))

50381


In [6]:
messages_bow = bow_transformer.transform(messages['message'])
print('Shape of Sparse Matrix: ', messages_bow.shape)
#Prints out (# of rows, # of columns)

Shape of Sparse Matrix:  (5172, 50381)


TF/IDF

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(messages_bow)

#To transform the entier bag of words
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(5172, 50381)


#**Naive Bayes Classifier**

In [8]:
from sklearn.naive_bayes import MultinomialNB
#using naive approch to classify our data
X_train = messages_tfidf[1:3000]
y_train = messages['label_num'][1:3000]

spam_detect_model = MultinomialNB().fit(X_train, y_train)

##Eval of Naive Approch

In [10]:
from sklearn.metrics import classification_report
X_test = messages_tfidf[3000:3300]
y_test = messages['label_num'][3000:3300]

all_pred = spam_detect_model.predict(X_test)
print(classification_report(y_test, all_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94       221
           1       1.00      0.63      0.78        79

    accuracy                           0.90       300
   macro avg       0.94      0.82      0.86       300
weighted avg       0.91      0.90      0.90       300



Stacked LSTM Structure

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming X_tfidf is your TF-IDF matrix and y are the corresponding labels
y = messages['label_num']
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(messages_tfidf, y, test_size=0.2, random_state=42)

# Define the number of timesteps
num_timesteps = 1  # You can adjust this based on your preference

# Flatten the TF-IDF matrix along the feature axis
X_train_flattened = np.reshape(X_train, (X_train.shape[0], -1))
X_test_flattened = np.reshape(X_test, (X_test.shape[0], -1))

# Define LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_flattened.shape[1],)),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_flattened, y_train, epochs=10, batch_size=32, validation_data=(X_test_flattened, y_test))


ValueError: Input 0 of layer "lstm_21" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 50381)

#**Using BERT Encoder**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
messages = [line.rstrip() for line in open('spam_ham_dataset.csv')]
print(len(messages))

# # making dataframe
# df = pd.read_csv("spam_ham_dataset.csv")

# # output the dataframe
# print(df.head())

messages = pd.read_csv('spam_ham_dataset.csv', sep=',', names=['Ignore', 'Label', 'message', 'label_num'])
messages.head()

101594


,Ignore,Label,message,label_num
0,NaN,label,text,label_num
1,605.0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
2,2349.0,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
3,3624.0,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
4,4685.0,spam,"Subject: photoshop , windows , office . cheap ...",1


# **Passing BERT Embedings to a stacked LSTM**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
# Assuming df is your pandas DataFrame with a 'text' column and a 'label' column
X_train = messages['message'][1:3000].tolist()

y_train = messages['label_num'][1:3000].values
for i in range(len(y_train)):
    y_train[i] = int(y_train[i])

y_train = y_train.tolist()

# Step 1: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_tokens = tokenizer(X_train, padding=True, truncation=True, return_tensors='tf', max_length=128)

# Step 2: Word Embedding
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
X_embeddings = bert_model(X_tokens)[1]

X_reshaped = tf.expand_dims(X_embeddings, axis=1)  # Adding a time step dimension

print(X_reshaped.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture

(2999, 1, 768)


**Training Using BERT tokenizer**

In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(1,768)))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Step 5: Compile the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Step 6: Train the Model
batch_size = 10
epochs = 15
model.fit(X_reshaped, np.array(y_train), batch_size=batch_size, epochs=epochs, validation_split=0.2)


Epoch 1/15
270/270 [==============================] - 8s 9ms/step - loss: 0.4781 - accuracy: 0.7814 - val_loss: 0.3416 - val_accuracy: 0.8833
Epoch 2/15
270/270 [==============================] - 2s 8ms/step - loss: 0.3190 - accuracy: 0.8714 - val_loss: 0.2233 - val_accuracy: 0.9200
Epoch 3/15
270/270 [==============================] - 2s 7ms/step - loss: 0.2255 - accuracy: 0.9144 - val_loss: 0.3406 - val_accuracy: 0.8500
Epoch 4/15
270/270 [==============================] - 2s 7ms/step - loss: 0.1975 - accuracy: 0.9192 - val_loss: 0.1585 - val_accuracy: 0.9267
Epoch 5/15
270/270 [==============================] - 2s 7ms/step - loss: 0.1966 - accuracy: 0.9229 - val_loss: 0.1412 - val_accuracy: 0.9500
Epoch 6/15
270/270 [==============================] - 2s 7ms/step - loss: 0.1851 - accuracy: 0.9259 - val_loss: 0.1347 - val_accuracy: 0.9500
Epoch 7/15
270/270 [==============================] - 2s 7ms/step - loss: 0.1736 - accuracy: 0.9303 - val_loss: 0.1297 - val_accuracy: 0.9400
Epoch 

# **Testing Using Bert Tokenizer**

### Accuracy

In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import tensorflow as tf

X_test = messages['message'][3000:3300].tolist()

y_test = messages['label_num'][3000:3300].values
for i in range(len(y_test)):
    y_test[i] = int(y_test[i])

y_test = y_test.tolist()

# Step 1: Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_tokens = tokenizer(X_test, padding=True, truncation=True, return_tensors='tf', max_length=128)

# Step 2: Word Embedding
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
X_embeddings = bert_model(X_tokens)[1]

X_reshaped = tf.expand_dims(X_embeddings, axis=1)  # Adding a time step dimension

pred = model.predict(X_reshaped, verbose=0)

predicted = pred.tolist()

loss, accuracy = model.evaluate(X_reshaped, np.array(y_test))
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


for i in range(len(predicted)):
  predicted[i] = 1 if predicted[i][0] >= 0.5 else 0

actual = numpy.array(y_test)
predicted = numpy.array(predicted)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

10/10 [==============================] - 0s 3ms/step - loss: 0.1413 - accuracy: 0.9433
Test Loss: 0.14125043153762817, Test Accuracy: 0.9433333277702332


###Confusion Matrix

In [ ]:
True_positive =  0
True_negative = 0

False_positive = 0
False_negative = 0
for i in range(len(y_test)):
  if y_test[i] == 1 and predicted[i] == 1:
    True_positive += 1
  elif(y_test[i] == 0 and predicted[i] == 0):
    True_negative += 1

  elif(y_test[i] == 0 and predicted[i] == 1):
    False_positive += 1
  elif(y_test[i] == 1 and predicted[i] == 0):
    False_negative += 1

print(f"     True   False ")
print(f"P:\t{True_positive} \t {False_positive}")
print(f"N:\t{True_negative} \t {False_negative}")